In [1]:
import pandas as pd

class Contract:
    
    def __init__(self, wholesale_price, revenue_share, surplus_refund):
        self.wholesale_price = wholesale_price
        self.revenue_share = revenue_share
        self.surplus_refund = surplus_refund
        
        self.retail_price = 4250
        self.surplus_price = 2500
        self.factory_price = 2250
        
        self.demand_frame = pd.DataFrame({'Demand':range(6),
                                          'Probability':[.1, .15, .3, .2, .15, .1]})     
    
    def vendor_marginal_profit(self):
        return self.retail_price \
             - self.wholesale_price \
             - (1 - self.revenue_share)*self.retail_price
    
    def vendor_marginal_loss(self):
        return (self.wholesale_price - self.surplus_refund - self.surplus_price)
    
    def optimal_service(self):
        mp = self.vendor_marginal_profit()
        ml = self.vendor_marginal_loss()
        return mp/(mp+ml)
    
    def optimal_quantity(self):
        df = self.demand_frame
        df['Cumulative Demand'] = df['Probability'].cumsum()
        df['Cumulative Demand'] = df['Cumulative Demand'].shift(1)
        mask = df[df['Cumulative Demand'] <= self.optimal_service()]
        return mask['Demand'].max()
    
    def daily_view(self):
        quantity = self.optimal_quantity()
        self.demand_frame['Number Sold Apr-Aug'] = self.demand_frame['Demand'].clip(0, quantity)
        self.demand_frame['Surplus'] = (quantity - self.demand_frame['Demand']).clip(0, None)
        
        self.demand_frame['McCormick Revenue'] = self.demand_frame['Number Sold Apr-Aug']\
                                                 *self.retail_price\
                                                +self.demand_frame['Surplus']*self.surplus_price
        
        self.demand_frame['Surplus Refund'] = self.demand_frame['Surplus']\
                                             *self.surplus_refund
        
        self.demand_frame['Revenue Sharing'] = self.demand_frame['McCormick Revenue']\
                                              *(1 - self.revenue_share)
        
        self.demand_frame['McCormick Revenue'] = self.demand_frame['McCormick Revenue']\
                                                +self.demand_frame['Surplus Refund']
        
        self.demand_frame['McCormick Cost'] = quantity*self.wholesale_price\
                                             +self.demand_frame['Revenue Sharing']
        
        self.demand_frame['McCormick Profit'] =  self.demand_frame['McCormick Revenue'] \
                                                -self.demand_frame['McCormick Cost'] \
        
        self.demand_frame['Wholesaler Revenue'] = quantity*self.wholesale_price\
                                                 +self.demand_frame['Revenue Sharing']
        
        self.demand_frame['Wholesaler Cost'] = quantity*self.factory_price\
                                              +self.demand_frame['Surplus Refund']
        
        self.demand_frame['Wholesaler Profit'] =  self.demand_frame['Wholesaler Revenue'] \
                                                 -self.demand_frame['Wholesaler Cost']   
                                                    
        self.demand_frame['Supply Chain Revenue'] = self.demand_frame['McCormick Revenue']\
                                                   +self.demand_frame['Wholesaler Revenue']

        self.demand_frame['Supply Chain Cost'] = self.demand_frame['McCormick Cost']\
                                                +self.demand_frame['Wholesaler Cost']      
        
        self.demand_frame['Supply Chain Profit'] = self.demand_frame['McCormick Profit']\
                                                +self.demand_frame['Wholesaler Profit'] 
        
        self.demand_frame = self.demand_frame.drop(['Revenue Sharing',
                                                    'Surplus Refund',
                                                    'Cumulative Demand'],
                                                     axis = 1) 
        
        return self.demand_frame  
    
    def vendor_expected_profit(self):
        df = self.daily_view()
        return (df['McCormick Profit']*df['Probability']).sum()
    
    def wholesale_expected_profit(self):
        df = self.daily_view()
        return (df['Wholesaler Profit']*df['Probability']).sum() 
    
    def supply_chain_expected_profit(self):
        df = self.daily_view()
        return ((df['Supply Chain Profit'])*df['Probability']).sum()

The above initalizes a class model with the case's variables as inputs

Question 1

Since we are able to always sell our lawnmowers higher than the factory price there is always a profit, meaning we should produce as many as possible. So we have:

In [2]:
max_supply = pd.DataFrame({'Demand':range(6),
                           'Probability':[.1, .15, .3, .2, .15, .1],
                           'Surplus':range(5, -1, -1)})   

In [3]:
#profit of 250 from surplus sales
max_supply['Surplus Profit'] = max_supply['Surplus']*250

#profit of 2000 from season sales
max_supply['Regular Profit'] = max_supply['Demand']*2000

max_supply['Total Profit'] = max_supply['Surplus Profit'] + max_supply['Regular Profit']

In [4]:
max_supply

,Demand,Probability,Surplus,Surplus Profit,Regular Profit,Total Profit
0,0,0.10,5,1250,0,1250
1,1,0.15,4,1000,2000,3000
2,2,0.30,3,750,4000,4750
3,3,0.20,2,500,6000,6500
4,4,0.15,1,250,8000,8250
5,5,0.10,0,0,10000,10000


In [5]:
#maximum expected profit
(max_supply['Total Profit']*max_supply['Probability']).sum()

5537.5

Question 2

Initialize the three contract models with given variables:

In [6]:
traditional = Contract(wholesale_price = 3000,
                       revenue_share = 1,
                       surplus_refund = 0)

In [7]:
rev_share = Contract(wholesale_price = 2600,
                     revenue_share = .9,
                     surplus_refund = 0)

In [8]:
surplus_refund = Contract(wholesale_price = 3000,
                          revenue_share = 1,
                          surplus_refund = 400)

In [9]:
idx = ['Retail Price',
       'Wholesale Price',
       'Factory Price',
       'McCormick Revenue Share',
       'Surplus Refund',
       'McCormick MP',
       'McCormick ML',
       'McCormick Optimal Order Quantity',
       'McCormick Expected Profit',
       'Wholesaler Expected Profit',
       'Supply Chain Expected Profit']

In [10]:
#calculate question requirements

traditional_series = pd.Series([traditional.retail_price,
                                traditional.wholesale_price,
                                traditional.factory_price,
                                traditional.revenue_share,
                                traditional.surplus_refund,
                                traditional.vendor_marginal_profit(),
                                traditional.vendor_marginal_loss(),
                                traditional.optimal_quantity(), 
                                traditional.vendor_expected_profit(), 
                                traditional.wholesale_expected_profit(), 
                                traditional.supply_chain_expected_profit()], 
                                
                                name = 'Traditional',
                                index = idx)

rev_share_series = pd.Series([rev_share.retail_price,
                              rev_share.wholesale_price,
                              rev_share.factory_price,
                              rev_share.revenue_share,
                              rev_share.surplus_refund,
                              rev_share.vendor_marginal_profit(),
                              rev_share.vendor_marginal_loss(),
                              rev_share.optimal_quantity(), 
                              rev_share.vendor_expected_profit(), 
                              rev_share.wholesale_expected_profit(), 
                              rev_share.supply_chain_expected_profit()],
                            
                              name = 'Revenue Sharing',
                              index = idx)

surplus_refund_series = pd.Series([surplus_refund.retail_price,
                                   surplus_refund.wholesale_price,
                                   surplus_refund.factory_price,
                                   surplus_refund.revenue_share,
                                   surplus_refund.surplus_refund,
                                   surplus_refund.vendor_marginal_profit(),
                                   surplus_refund.vendor_marginal_loss(),
                                   surplus_refund.optimal_quantity(), 
                                   surplus_refund.vendor_expected_profit(), 
                                   surplus_refund.wholesale_expected_profit(), 
                                   surplus_refund.supply_chain_expected_profit()], 
                                   
                                   name = 'Surplus-Refund',
                                   index = idx)

df_list = [traditional_series, rev_share_series, surplus_refund_series]

In [11]:
#display answer
pd.concat(df_list, axis = 1)

,Traditional,Revenue Sharing,Surplus-Refund
Retail Price,4250.0,4250.00,4250.0
Wholesale Price,3000.0,2600.00,3000.0
Factory Price,2250.0,2250.00,2250.0
McCormick Revenue Share,1.0,0.90,1.0
Surplus Refund,0.0,0.00,400.0
McCormick MP,1250.0,1225.00,1250.0
McCormick ML,500.0,100.00,100.0
McCormick Optimal Order Quantity,3.0,5.00,5.0
McCormick Expected Profit,2175.0,2108.75,2807.5
Wholesaler Expected Profit,2250.0,3428.75,2730.0


In [12]:
traditional.daily_view()

,Demand,Probability,Number Sold Apr-Aug,Surplus,McCormick Revenue,McCormick Cost,McCormick Profit,Wholesaler Revenue,Wholesaler Cost,Wholesaler Profit,Supply Chain Revenue,Supply Chain Cost,Supply Chain Profit
0,0,0.10,0,3,7500,9000,-1500,9000,6750,2250,16500,15750,750
1,1,0.15,1,2,9250,9000,250,9000,6750,2250,18250,15750,2500
2,2,0.30,2,1,11000,9000,2000,9000,6750,2250,20000,15750,4250
3,3,0.20,3,0,12750,9000,3750,9000,6750,2250,21750,15750,6000
4,4,0.15,3,0,12750,9000,3750,9000,6750,2250,21750,15750,6000
5,5,0.10,3,0,12750,9000,3750,9000,6750,2250,21750,15750,6000


In [13]:
rev_share.daily_view()

,Demand,Probability,Number Sold Apr-Aug,Surplus,McCormick Revenue,McCormick Cost,McCormick Profit,Wholesaler Revenue,Wholesaler Cost,Wholesaler Profit,Supply Chain Revenue,Supply Chain Cost,Supply Chain Profit
0,0,0.10,0,5,12500,14250.0,-1750.0,14250.0,11250,3000.0,26750.0,25500.0,1250.0
1,1,0.15,1,4,14250,14425.0,-175.0,14425.0,11250,3175.0,28675.0,25675.0,3000.0
2,2,0.30,2,3,16000,14600.0,1400.0,14600.0,11250,3350.0,30600.0,25850.0,4750.0
3,3,0.20,3,2,17750,14775.0,2975.0,14775.0,11250,3525.0,32525.0,26025.0,6500.0
4,4,0.15,4,1,19500,14950.0,4550.0,14950.0,11250,3700.0,34450.0,26200.0,8250.0
5,5,0.10,5,0,21250,15125.0,6125.0,15125.0,11250,3875.0,36375.0,26375.0,10000.0


In [14]:
surplus_refund.daily_view()

,Demand,Probability,Number Sold Apr-Aug,Surplus,McCormick Revenue,McCormick Cost,McCormick Profit,Wholesaler Revenue,Wholesaler Cost,Wholesaler Profit,Supply Chain Revenue,Supply Chain Cost,Supply Chain Profit
0,0,0.10,0,5,14500,15000,-500,15000,13250,1750,29500,28250,1250
1,1,0.15,1,4,15850,15000,850,15000,12850,2150,30850,27850,3000
2,2,0.30,2,3,17200,15000,2200,15000,12450,2550,32200,27450,4750
3,3,0.20,3,2,18550,15000,3550,15000,12050,2950,33550,27050,6500
4,4,0.15,4,1,19900,15000,4900,15000,11650,3350,34900,26650,8250
5,5,0.10,5,0,21250,15000,6250,15000,11250,3750,36250,26250,10000
